# recomendation system

In [35]:
# import pandas library 
import pandas as pd 

In [36]:
#import dataset 
books= pd.read_csv("C:/Users/Mandar/Desktop/data/assignments/recomendation system/books.csv",header=0,encoding='ISO-8859-1',names=['unname','Users','book_title','book_author','publisher','ratings'])

In [37]:
books.head()


,unname,Users,book_title,book_author,publisher,ratings
0,1,1,Classical Mythology,Mark P. O. Morford,Oxford University Press,0
1,2,2,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,5
2,3,3,Decision in Normandy,Carlo D'Este,HarperPerennial,0
3,4,4,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,Farrar Straus Giroux,3
4,5,5,The Mummies of Urumchi,E. J. W. Barber,W. W. Norton &amp; Company,6


In [38]:
books.shape

(5000, 6)

In [39]:
books.columns

Index(['unname', 'Users', 'book_title', 'book_author', 'publisher', 'ratings'], dtype='object')

In [40]:
books01 = books.iloc[0:4000,1:]

In [41]:
books01.head()

,Users,book_title,book_author,publisher,ratings
0,1,Classical Mythology,Mark P. O. Morford,Oxford University Press,0
1,2,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,5
2,3,Decision in Normandy,Carlo D'Este,HarperPerennial,0
3,4,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,Farrar Straus Giroux,3
4,5,The Mummies of Urumchi,E. J. W. Barber,W. W. Norton &amp; Company,6


In [42]:
books01 = books01[~(books01 == 0).any(axis=1)] #remove 0 rating from rows

In [43]:
books01.head()

,Users,book_title,book_author,publisher,ratings
1,2,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,5
3,4,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,Farrar Straus Giroux,3
4,5,The Mummies of Urumchi,E. J. W. Barber,W. W. Norton &amp; Company,6
6,7,What If?: The World's Foremost Military Histor...,Robert Cowley,Berkley Publishing Group,8
7,8,PLEADING GUILTY,Scott Turow,Audioworks,6


In [44]:
books01.book_title

1                                            Clara Callan
3       Flu: The Story of the Great Influenza Pandemic...
4                                  The Mummies of Urumchi
6       What If?: The World's Foremost Military Histor...
7                                         PLEADING GUILTY
                              ...                        
3992            We Are Still Married: Stories and Letters
3993    The Illustrated Discovery Journal : Creating a...
3994    Creating a Charmed Life : Sensible, Spiritual ...
3995    Eat Mangoes Naked : Finding Pleasure Everywher...
3997    All the Joy You Can Stand: 101 Sacred Power Pr...
Name: book_title, Length: 1692, dtype: object

In [45]:
books01.book_author

1        Richard Bruce Wright
3            Gina Bari Kolata
4             E. J. W. Barber
6               Robert Cowley
7                 Scott Turow
                ...          
3992         Garrison Keillor
3993     Sarah Ban Breathnach
3994           Victoria Moran
3995                     Sark
3997    Debrena Jackson Gandy
Name: book_author, Length: 1692, dtype: object

In [46]:
books01.publisher

1            HarperFlamingo Canada
3             Farrar Straus Giroux
4       W. W. Norton &amp; Company
6         Berkley Publishing Group
7                       Audioworks
                   ...            
3992                 Penguin Books
3993                  Warner Books
3994            HarperSanFrancisco
3995                      Fireside
3997       Three Rivers Press (CA)
Name: publisher, Length: 1692, dtype: object

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [48]:
books01['book_author'].isnull().sum()

0

In [49]:
tfid= TfidfVectorizer(stop_words="english")    #taking stop words from tfid vectorizer

In [50]:
tfid_matrix = tfid.fit_transform(books01.publisher)

In [51]:
tfid_matrix.shape 

(1692, 655)

In [52]:
# Lets Find the Similarity Score
from sklearn.metrics.pairwise import linear_kernel

In [53]:
cosine_sim_matrix = linear_kernel(tfid_matrix,tfid_matrix)

In [54]:
books01_index = pd.Series(books01.index,index=books01['book_title']).drop_duplicates()

In [55]:
books01_index['Clara Callan']

1

In [56]:
def get_book_recommendations(Name,topN):
    
   
    topN = 10
    # The book index using its title 
    title_id = books01_index[Name]
    
    # Getting the pair wise similarity score for all the Books with that 
    # Book
    cosine_scores = list(enumerate(cosine_sim_matrix[title_id]))
    
    # Sorting the cosine_similarity scores based on scores 
    cosine_scores = sorted(cosine_scores,key=lambda x:x[1],reverse = True)
    
    # Get the scores of top 10 most similar Books 
    cosine_scores_10 = cosine_scores[0:topN+1]
    
    # Getting the Book index 
    books01_idx  =  [i[0] for i in cosine_scores_10]
    books01_scores =  [i[1] for i in cosine_scores_10]
    
    # Similar Books and scores
    book_similar_show = pd.DataFrame(columns=["name","Score"])
    book_similar_show["name"] = books01.loc[books01_idx,"book_title"]
    book_similar_show["Score"] = books01_scores
    book_similar_show.reset_index(inplace=True)  
    book_similar_show.drop(["index"],axis=1,inplace=True)
    print (book_similar_show)

In [59]:
get_book_recommendations("PLEADING GUILTY",topN=15) 

                                                 name     Score
0                                     PLEADING GUILTY  1.000000
1                   My Father, Dancing (Harvest Book)  1.000000
2   Die Grenzen des Geschlechts: Anleitungen zum S...  0.558703
3                                        The Carousel  0.558703
4               Anne Frank: The Diary of a Young Girl  0.558703
5                                                 NaN  0.558703
6                                                 NaN  0.405720
7   All That Remains (Kay Scarpetta Mysteries (Pap...  0.358259
8   Flu: The Story of the Great Influenza Pandemic...  0.181945
9                                                 NaN  0.181945
10                                                NaN  0.181945


# So we can Recommend the book "My Father, Dancing(Harvest Book) to the Users based on Publsher of Pleading Guilty

In [58]:
get_book_recommendations("Clara Callan",topN=15)

                                                 name     Score
0                                        Clara Callan  1.000000
1                                      Ralph S. Mouse  1.000000
2      XIII, tome 13, L'enquÃ?Âªte : the XIII mystery  1.000000
3                       Eeyore's Little Book of Gloom  1.000000
4                                                 NaN  1.000000
5                                                 NaN  1.000000
6                                           Die Nadel  1.000000
7                              The Soulbane Stratagem  0.928365
8                                                 NaN  0.928365
9   Songs in Ordinary Time (Oprah's Book Club (Pap...  0.803688
10                                                NaN  0.000000


# we recommend Books Ralph S. Mouse and 3 more to the Users based on the Publisher of Clara Callan